In [1]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import FAISS
from dotenv import load_dotenv

In [2]:
load_dotenv()
persist_directory = "chroma_db"

# settings = Settings(persist_directory=persist_directory)


In [3]:
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

llm = ChatOpenAI(
    model="gpt-4o-mini-2024-07-18",
    temperature=0
)


C:\Users\dario\AppData\Local\Temp\ipykernel_22500\34073894.py:3: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


In [4]:
pdf_dir = "data"
pdf_files = [os.path.join(pdf_dir, f) for f in os.listdir(pdf_dir) if f.endswith(".pdf")]

docs = []
for path in pdf_files:
    print(f" → Carico: {path}")
    loader = PyPDFLoader(path)
    docs.extend(loader.load())

# --- Split dei documenti ---
print("✂️  Split in chunk...")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(docs)
print(f"Totale chunk generati: {len(texts)}")

# --- Costruzione Vector DB in memoria con FAISS ---
print("🚀 Costruisco il vector store FAISS...")
vector_db = FAISS.from_documents(texts, embeddings)

# --- Retriever per future query ---
retriever = vector_db.as_retriever()

print("✅ Ingestione completata con successo (FAISS, in RAM).")

 → Carico: data\[RL1]IntroRL-1-8.pdf
✂️  Split in chunk...
Totale chunk generati: 8
🚀 Costruisco il vector store FAISS...
✅ Ingestione completata con successo (FAISS, in RAM).


In [5]:
query = "How are graphs used to improve reasoning or retrieval in GraphRAG?"

retrieved_docs = retriever.invoke(query)

for i, doc in enumerate(retrieved_docs):
    print(f"\n--- Document {i + 1} ---")
    print(doc.page_content[:1000])  # Mostra i primi 1000 caratteri



--- Document 1 ---
Intelligenza Artificiale
a.a. 2024/2025
Reinforcement Learning

--- Document 2 ---
Intelligenza Artificiale                                                                                                     a.a. 2024/2025
Problemi con l’apprendimento 
supervisionato
 Cosa c'è di sbagliato nell'apprendimento supervisionato?
 Un agente di apprendimento supervisionato apprende 
osservando passivamente l'esempio coppie ingresso/uscita 
fornite da un “insegnante”
 Possiamo addestrare l'agente di scacchi con l'apprendimento 
supervisionato?
 Dati: esempi di posizioni scacchistiche (ognuna etichettata con la 
corretta mossa)
 Training data disponibili per i vincitori: 108
 Lo spazio di tutte le possibili posizioni degli scacchi: 1040
 Gli scacchi non possono essere risolti con l'apprendimento 
supervisionato

--- Document 3 ---
Intelligenza Artificiale                                                                                                     a.a. 2024/2025

In [6]:
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True  # utile per vedere i documenti usati
)

# Fai una domanda
query = "Che cos'è il Reinforcement Learning?"
result = qa_chain({"query": query})

# Mostra la risposta
print("\n🧠 Risposta generata:")
print(result["result"])

# Mostra i documenti di supporto (opzionale)
print("\n📄 Documenti utilizzati:")
for doc in result["source_documents"]:
    print(f"• {doc.metadata.get('source', 'N/A')}:\n{doc.page_content[:300]}\n")

C:\Users\dario\AppData\Local\Temp\ipykernel_22500\2727519620.py:10: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": query})



🧠 Risposta generata:
Il Reinforcement Learning (RL) è un paradigma di apprendimento automatico in cui un agente apprende a prendere decisioni attraverso l'interazione con un ambiente. A differenza di altri paradigmi di machine learning, nel RL non c'è supervisione diretta; l'agente riceve solo ricompense o penalità in base alle azioni che compie. Il feedback non è immediato e il tempo gioca un ruolo importante, poiché le azioni dell'agente influenzano le informazioni future che riceve. Questo approccio è particolarmente utile per problemi sequenziali e per situazioni in cui l'apprendimento deve avvenire in modo continuo.

📄 Documenti utilizzati:
• data\[RL1]IntroRL-1-8.pdf:
Intelligenza Artificiale                                                                                                     a.a. 2024/2025
Caratteristiche del RL
 Cosa rende il Reinforcement Learning diverso dagli altri 
paradigmi di Machine Learning?
 Non c’è alcuna supervisione, solo ricompense

• data\[RL1]In